<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-09 14:28:50
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 15.60 K (0.11%)
Current PnL: -13.06 L (-9.1%)
CY Booked + Current PnL: -6.10 L (-4.25%)
-------------------
Total profit:  4.89 L
Total loss:  -17.95 L
-------------------
Total Booked + Current PnL: 20.88 L (17.92%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.55 L (52.21%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85508.0,-9808.0,9851.0,4.08,-10.29,11.52,0.04,157.0,-1.00,0.62,68.65,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-17.63,H-LC,90.62,112222.0,8656.0,12053.0,-1.10,8.36,10.74,20.00,32.0,0.72,0.82,23.61,X40N,BTT,HEALTHCARE
71,SBIN,760.30,863.00,-11.51,M-LC,33.33,214981.0,13502.0,13716.0,-0.17,6.70,6.38,13.51,77.0,0.98,1.57,20.14,XY25,NTT,BANKS
39,HINDALCO,651.95,761.55,-21.55,H-LC,72.92,107984.0,3672.0,13865.0,-1.56,3.52,12.84,16.81,27.0,0.26,0.79,20.13,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,47.02,M-SC,23.96,127818.0,-14661.0,15287.0,0.45,-10.29,11.96,0.44,247.0,-0.96,0.93,50.86,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-32.52,M-MC,6.25,178223.0,-1585.0,113991.0,-0.92,-0.88,63.96,62.52,173.0,-0.01,1.30,32.91,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,40.62,151900.0,987.0,59469.0,0.92,0.65,39.15,40.06,103.0,0.02,1.11,24.24,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,MEDANTA,1064.65,1486.00,16.12,X-SC,87.50,247561.0,42084.0,39238.0,5.30,20.48,15.85,39.58,65.0,1.07,1.81,32.66,XY24,NTT,HEALTHCARE
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85508.0,-9808.0,9851.0,4.08,-10.29,11.52,0.04,157.0,-1.00,0.62,68.65,OX40N,NTT,MISC
32,ENRIN,1377.95,3146.20,NaN,NaN,95.83,78655.0,44206.0,0.0,3.26,128.32,0.00,128.32,274.0,inf,0.57,20.28,AR,ATH,ELECTRICAL
54,KPIGREEN,497.21,731.84,27.90,H-SC,51.04,130750.0,5453.0,53673.0,2.30,4.35,41.05,47.19,143.0,0.10,0.95,63.58,MH,ATH,POWER
55,LALPATHLAB,2804.19,3545.00,-29.81,H-SC,61.46,161618.0,7388.0,33358.0,2.03,4.79,20.64,26.42,127.0,0.22,1.18,27.45,X40N,NTT,HEALTHCARE


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,19.79,48837.0,7582.0,3223.0,-4.27,18.38,6.60,26.19,272.0,2.35,0.36,14.98,XY24,ATH,IT
88,UNIONBANK,123.87,163.00,-6.01,M-LC,5.21,163955.0,23115.0,21380.0,-4.03,16.41,13.04,31.59,88.0,1.08,1.20,42.42,XY24,NTT,BANKS
90,VALIANTORG,512.64,838.00,-353.90,H-SC,10.42,107900.0,-25386.0,109982.0,-3.15,-19.05,101.93,63.47,151.0,-0.23,0.79,80.87,XR,NTT,CHEMICALS
41,HINDZINC,514.80,730.22,12.55,H-LC,1.04,84880.0,-18080.0,61165.0,-2.71,-17.56,72.06,41.85,24.0,-0.30,0.62,11.71,X5K,ATH,METALS
0,5PAISA,564.75,565.00,148.80,H-SC,18.75,134353.0,-52579.0,52666.0,-2.49,-28.13,39.20,0.04,149.0,-1.00,0.98,28.53,OX40N,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,23.96,127818.0,-14661.0,15287.0,0.45,-10.29,11.96,0.44,247.0,-0.96,0.93,50.86,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-218.54,H-SC,98.96,85508.0,-9808.0,9851.0,4.08,-10.29,11.52,0.04,157.0,-1.00,0.62,68.65,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-29.37,M-SC,25.00,150676.0,-20465.0,20417.0,-0.33,-11.96,13.55,-0.03,191.0,-1.00,1.10,8.29,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,92.71,148924.0,-25635.0,25674.0,0.06,-14.69,17.24,0.02,133.0,-1.00,1.09,29.14,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.98,X-MC,9.38,86114.0,-16538.0,16534.0,-0.33,-16.11,19.20,-0.00,59.0,-1.00,0.63,30.90,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.02,X-LC,31.25,263804.0,-32130.0,126046.0,-0.71,-10.86,47.78,31.73,1.0,-0.25,1.93,5.59,X40,BTT,IT
48,INFY,1461.46,2275.00,-8.83,X-LC,73.96,292164.0,30563.0,115054.0,-0.40,11.68,39.38,55.67,2.0,0.27,2.13,18.50,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.30,X-LC,62.50,271320.0,973.0,33644.0,1.25,0.36,12.40,12.81,5.0,0.03,1.98,13.47,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.40,X-LC,66.67,269307.0,40074.0,27200.0,0.27,17.48,10.10,29.35,10.0,1.47,1.97,27.96,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-4.59,X-LC,15.62,80722.0,-29450.0,118524.0,-2.06,-26.73,146.83,80.85,11.0,-0.25,0.59,18.89,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3146.2,NaN,NaN,95.83,78655.0,44206.0,0.0,3.26,128.32,0.00,128.32,274.0,inf,0.57,20.28,AR,ATH,ELECTRICAL
59,MEDANTA,1064.65,1486.0,16.12,X-SC,87.50,247561.0,42084.0,39238.0,5.30,20.48,15.85,39.58,65.0,1.07,1.81,32.66,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.10,M-MC,35.42,228957.0,3995.0,163658.0,-1.55,1.78,71.48,74.53,189.0,0.02,1.67,32.93,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.84,27.90,H-SC,51.04,130750.0,5453.0,53673.0,2.30,4.35,41.05,47.19,143.0,0.10,0.95,63.58,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.09,H-SC,26.04,116928.0,14924.0,30097.0,1.31,14.63,25.74,44.14,126.0,0.50,0.85,33.25,X40N,ATH,MISC
95,WIPRO,243.46,420.00,-3.31,M-LC,85.42,165850.0,14905.0,94551.0,-0.80,9.87,57.01,72.51,71.0,0.16,1.21,14.24,XR,NTT,IT
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181060.0,16390.0,52688.0,0.03,9.95,29.10,41.95,124.0,0.31,1.32,18.16,X40N,ATH,AC


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3146.20,NaN,NaN,95.83,78655.0,44206.0,0.0,3.26,128.32,0.00,128.32,274.0,inf,0.57,20.28,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-39.82,M-SC,41.67,198553.0,26932.0,135135.0,1.33,15.69,68.06,94.43,201.0,0.20,1.45,46.54,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.31,M-LC,85.42,165850.0,14905.0,94551.0,-0.80,9.87,57.01,72.51,71.0,0.16,1.21,14.24,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.52,H-SC,77.08,136470.0,13134.0,120626.0,0.83,10.65,88.39,108.45,123.0,0.11,1.00,35.38,AR,ATH,MISC
54,KPIGREEN,497.21,731.84,27.90,H-SC,51.04,130750.0,5453.0,53673.0,2.30,4.35,41.05,47.19,143.0,0.10,0.95,63.58,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.05,NaN,NaN,58.33,18845.0,-30748.0,0.0,1.57,-62.00,0.00,-62.00,270.0,-inf,0.14,26.38,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,67.71,65357.0,-48192.0,88245.0,1.46,-42.44,135.02,35.27,265.0,-0.55,0.48,59.96,XR,NTT,HOTELS
32,ENRIN,1377.95,3146.20,NaN,NaN,95.83,78655.0,44206.0,0.0,3.26,128.32,0.00,128.32,274.0,inf,0.57,20.28,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7183.33,M-SC,82.29,79927.0,-13883.0,90501.0,0.71,-14.80,113.23,81.67,245.0,-0.15,0.58,54.78,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85508.0,-9808.0,9851.0,4.08,-10.29,11.52,0.04,157.0,-1.00,0.62,68.65,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-16.18,L-SC,100.00,168651.0,26809.0,29767.0,-0.54,18.90,17.65,39.88,262.0,0.90,1.23,70.91,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-218.54,H-SC,98.96,85508.0,-9808.0,9851.0,4.08,-10.29,11.52,0.04,157.0,-1.00,0.62,68.65,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.43,X-SC,97.92,100065.0,-898.0,95432.0,-1.22,-0.89,95.37,93.63,67.0,-0.01,0.73,49.05,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181060.0,16390.0,52688.0,0.03,9.95,29.10,41.95,124.0,0.31,1.32,18.16,X40N,ATH,AC
32,ENRIN,1377.95,3146.20,NaN,NaN,95.83,78655.0,44206.0,0.0,3.26,128.32,0.00,128.32,274.0,inf,0.57,20.28,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.03
1,25,42.04
2,50,70.69


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    33.86
MC    21.57
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.01
X40      12.95
XY25     12.10
X40N     11.95
XR        9.98
AR        8.18
OX40N     8.16
MH        1.67
X5K       1.41
X200      1.40
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.50
M-SC    15.86
H-LC    14.09
X-LC    12.91
H-MC     9.58
X-MC     7.61
M-LC     5.82
X-SC     4.54
M-MC     4.03
L-SC     2.62
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.30,0.03,33.40
IT,12.05,-10.15,64.29
FINANCE,8.51,-11.86,52.45
BANKS,7.65,-8.83,53.23
MISC,5.87,-19.52,75.18
PAINTS,5.71,-11.67,33.67
ELECTRICAL,5.46,1.38,45.25
HEALTHCARE,4.74,0.67,28.35
AUTO,4.30,-14.83,65.32


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2645703.0,26
XR,1090419.0,13
AR,1027940.0,10
X40,682043.0,10
X40N,526618.0,11
XY25,463197.0,8
OX40N,311549.0,11
SR,178519.0,2
MH,85841.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1974385.0,20
M-SC,1769355.0,21
H-LC,611702.0,12
X-LC,544081.0,8
H-MC,523007.0,8
M-MC,417515.0,3
X-MC,411698.0,6
X-SC,296299.0,5
M-LC,264278.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      887526.0      6
M-SC       XY24      805904.0      7
H-SC       AR        524156.0      3
M-SC       XR        337358.0      4
M-MC       XY24      303524.0      2
H-MC       XY24      297850.0      3
X-LC       X40       276880.0      3
H-SC       XR        217682.0      3
           X40N      203157.0      4
H-LC       X40N      193926.0      4
M-SC       AR        187393.0      2
           OX40N     167146.0      5
M-LC       XY24      156011.0      2
X-MC       X40       155716.0      2
H-LC       X40       154718.0      3
L-SC       XR        150527.0      2
X-SC       XR        121951.0      1
M-SC       XY25      119103.0      1
X-LC       AR        118524.0      1
M-MC       XR        113991.0      1
X-MC       XY25      102663.0      1
X-LC       XY25       98856.0      3
X-SC       SR         95432.0      1
M-LC       XR         94551.0      1
H-SC       OX40N      88191.0      3
X-MC       XY24       83139.0      1
M-SC       SR         83087.0      1
H-MC       XY25       82087.0      1
H-LC       AR         79917.0      1
           X5K        75030.0      2
M-SC       X40        69364.0      1
H-LC       X200       68590.0      1
H-MC       AR         59469.0      1
L-SC       AR         58481.0      1
L-MC       XR         54359.0      1
H-SC       MH         53673.0      1
X-MC       X40N       53646.0      1
X-LC       X40N       49821.0      1
L-LC       XY25       46772.0      1
X-SC       OX40N      39678.0      2
H-LC       XY24       39521.0      1
X-SC       XY24       39238.0      1
H-MC       MH         32168.0      1
L-SC       XY24       29767.0      1
H-MC       X40N       26068.0      1
           X40        25365.0      1
X-MC       OX40N      16534.0      1
M-LC       XY25       13716.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
